# Team 3 Case Study 1 : Collecting Data from Twitter

Due Date: 2/8/2021

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Twitter_bird_logo_2012.svg/220px-Twitter_bird_logo_2012.svg.png">

**Team 3 Members:** 

    Kate Sincaglia
    
    Mitchell Sirois
    
    Bryson Tang
    
    Jyalu Wu

# Problem 1 (20 points): Sampling Twitter Data with either Search or Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter API to sample a collection of tweets about this topic. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [3]:
import twitter
import json
from collections import Counter
from prettytable import PrettyTable

In [4]:
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    
    CONSUMER_KEY = 'oVek8jmhI4sz8SbKnT1e6UsIh'
    CONSUMER_SECRET ='3eGB6ip82K3XowIDEmz9JXIbRc2uFSlHvjPJ8Lmkf3HVIdlmhM'
    OAUTH_TOKEN = '571213367-H0r2WfySE7L4vgUJHv4YLW2vZGLa7TEPP7iWsTNu'
    OAUTH_TOKEN_SECRET = 'uKcYixkDRaCUvaP2oqH9a9fIaIB9nf4NKz32iqo2t7B7K'

    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

twitter_api = oauth_login()

#----------------------------------------------

q = 'outage'

count = 100

# Import unquote to prevent URL encoding errors in next_results
from urllib.parse import unquote

search_results = twitter_api.search.tweets(q=q, count=count)

statuses = search_results['statuses']

for _ in range(5): #takes count 5 times, 100*5 = 500
    print('Length of statuses', len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError as e: # No more results when next_results doesn't exist
        break

    # Create a dictionary from next_results, which has the following form:
    kwargs = dict([ kv.split('=') for kv in unquote(next_results[1:]).split("&") ])

    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

# Show one sample search result by slicing the list...
print(json.dumps(statuses[0], indent=1)) 

### Report some statistics about the tweets you collected 

When deciding on our topic of interest our team was inspired by the recent storm that hit the Eastern US and chose to look into power outages. 
To this end we collected 500 tweets that included the word 'outage' in them.

# Problem 2 (20 points): Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [21]:
import cloudpickle
import nltk
# import matplotlib.pyplot as plt
from prettytable import PrettyTable

# pickling/saving data
f = open('outages.pickle', 'wb')
cloudpickle.dump(statuses, f)
f.close()

# getting only the words in the tweets, clean them
words = []
result_set = set()

for tweet in statuses:
    text = tweet['text']
    text = text.lower()
    dirty_words = [ w for w in text.split() ]
    dirty_words = [word.strip('.,!;()[]:') for word in dirty_words]
    dirty_words = [word.replace("'s", '') for word in dirty_words]
    words += dirty_words

# frequency distribution
freq_dist = nltk.FreqDist(words)

# top 30 words
top30 = freq_dist.most_common(30)

# table of top 30 words
x = PrettyTable()
x.title = 'Top 30 Words'
x.field_names = ['Word', 'Count']
x.add_rows(top30)
print(x)

**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [ ]:
def get_tweets_rt_count(data):
    tweets = []
    rt_counts = []

    for tweet in data:
        tweets.append(tweet['text'])
        rt_counts.append(tweet['retweet_count'])

    return list(set(zip(tweets, rt_counts)))

In [ ]:

"""
    Returns the 2nd item in a tuple
"""
def rt_count(tup):
    return tup[1]

In [3]:
from tables import *
import numpy as np

# retrieve data/unpickle
pickle_off = open("outages.pickle","rb")
outages = cloudpickle.load(pickle_off)

# get only the tweets and retweet count
tweets_rts = get_tweets_rt_count(outages)

# sort by retweet count
tweets_rts.sort(reverse=True,key=rt_count)

# top 10 table
top10 = tweets_rts[:10]

x = PrettyTable()
x.title = 'Top 10 Most Popular Tweets'
x.field_names = ['Tweet', 'Retweet Count']
x.add_rows(top10)
x._max_width = {"Tweet" : 50, "Retweet Count" : 25}
x.hrules = 1
print(x)

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [23]:
#gather up all of the mentioned users and hashtags

hashtags = [ hashtag['text']
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

mentioned_user = [user_mention['screen_name']
                 for status in statuses
                     for user_mention in status['entities']['user_mentions']] #this collects all user mentions

#count how many times each hashtag is used and user is mentioned

for item in [hashtags, mentioned_user]:
    c = Counter(item)
    print(c.most_common()[:10]) # top 10
    print()
    
#format the data into a pretty table

for label, data in (('Hashtags', hashtags),
                    ('Mentioned Users', mentioned_user)):
    pt = PrettyTable(field_names=[label, 'Count'])
    c = Counter(data)
    [ pt.add_row(kv) for kv in c.most_common()[:10] ]
    pt.align[label], pt.align['Count'] = 'l', 'r' # Set column alignment
    print(pt)

[('WellingtonOPP', 9), ('WellingtonCounty', 8), ('Outage', 6), ('Guelph', 6), ('cryptocurrency', 4), ('Mets', 4), ('BREAKING', 3), ('JoburgUpdates', 2), ('StoneyCreek', 2), ('poweroutage', 2)]

[('JustRyCole', 48), ('CityPowerJhb', 48), ('BauerOutage', 11), ('PHEDconnect', 9), ('OPP_WR', 8), ('opg', 8), ('PSLToFlushing', 7), ('virginmedia', 6), ('GuelphHydro', 6), ('SkyHelpTeam', 6)]

+------------------+-------+
| Hashtags         | Count |
+------------------+-------+
| WellingtonOPP    |     9 |
| WellingtonCounty |     8 |
| Outage           |     6 |
| Guelph           |     6 |
| cryptocurrency   |     4 |
| Mets             |     4 |
| BREAKING         |     3 |
| JoburgUpdates    |     2 |
| StoneyCreek      |     2 |
| poweroutage      |     2 |
+------------------+-------+
+-----------------+-------+
| Mentioned Users | Count |
+-----------------+-------+
| JustRyCole      |    48 |
| CityPowerJhb    |    48 |
| BauerOutage     |    11 |
| PHEDconnect     |     9 |
| OPP_WR  

# Problem 3 (20 points): Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [121]:
import time
from prettytable import PrettyTable

# Get followers
friend_cursor = -1
follower_cursor = -1

user = "TweetWorcester"

number_of_users_to_collect = 300

searches = 0

followers = []
friends = []
while friend_cursor != 0 and follower_cursor != 0 and number_of_users_to_collect != 0:
    try:
        count = 200
        if number_of_users_to_collect < 200:
            count = number_of_users_to_collect
        
        if follower_cursor != 0:
            res_follower = twitter_api.followers.list(screen_name=user, 
                                                      cursor=follower_cursor, 
                                                      count=count)
            
            followers += [{"screen_name": u["screen_name"], "id": u["id"]} for u in res_follower['users']]
            follower_cursor = res_follower["next_cursor"]
                        
        if friend_cursor != 0:            
            res_friends = twitter_api.friends.list(screen_name=user, 
                                                   ursor=friend_cursor, 
                                                   count=count)
            
            friends += [{"screen_name": u["screen_name"], "id": u["id"]}for u in res_friends['users']]
            friend_cursor = res_friends["next_cursor"]
        
        searches += 1
        number_of_users_to_collect -= count
    except twitter.api.TwitterHTTPError:
        print("Waiting...")
        rates = twitter_api.application.rate_limit_status()
        
        time_left = rates["resources"]["followers"]["/followers/list"]["reset"] - time.time()
        
        time.sleep(time_left + 2)
    
    


In [115]:
def generate_user_table(data, title, length=20):
    table = PrettyTable(["Screen Name", "Id"])
    table.title = title
    
    if len(data) < length:
        length = len(data)
        
    for i in range(length):
        row = data[i]
        table.add_row([row["screen_name"], row["id"]])
    
    return table

In [102]:
time.time()

1612548034.287955

In [118]:
from prettytable import PrettyTable

friends_table  = generate_user_table(friends, "Friends")
followers_table  = generate_user_table(followers, "Followers")

print(friends_table)
print(followers_table)

+---------------------------------------+
|                Friends                |
+-----------------+---------------------+
|   Screen Name   |          Id         |
+-----------------+---------------------+
|     tedlieu     |       21059255      |
|      wsbmod     | 1282418324228337665 |
|  OldMemeArchive | 1349856571988258826 |
|      POTUS      | 1349149096909668363 |
|    nookipedia   |      213478688      |
|     Pokemon     |       96879107      |
|   staceyabrams  |      216065430      |
|     jschlatt    |      4694011844     |
| ProgressBar202_ |      4685749051     |
|  LeftAccidental | 1303873858936737793 |
|   mcblockdaily  | 1324477660182007810 |
|   StrangestMp4  | 1194413925569159169 |
|  CallMeCarsonYT |      2349536690     |
|  MoistCr1TiKaL  |      559229566      |
|    jacksfilms   |       9989862       |
|    robber0540   |      3086280070     |
|   hasanthehun   |      326756275      |
|   KamalaHarris  |       30354991      |
|     JoeBiden    |        939091 

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [122]:
mutual = []
for u in followers:
    for r in friends:
        if u['id'] == r["id"]:
            mutual.append(u)
            
mutual_table  = generate_user_table(mutual, "Mutual")

print(mutual_table)

+-----------------------------+
|            Mutual           |
+----------------+------------+
|  Screen Name   |     Id     |
+----------------+------------+
| ashleybaldw1n  | 3133724330 |
| srastevensb201 | 885519948  |
|    ralphpig    | 860254500  |
|    tang_meg    |  11726682  |
|    mmbtang     |  11699752  |
+----------------+------------+


# Problem 4 (20 points): Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [74]:
import pandas as pd
import numpy as np

q = 'snow AND outage'
#q = "andrew"
# lat = "42.2743"
# long = "-71.8081"
# rad = "50mi"
# geocode = lat + "," + long + "," + rad

# remaining_searches = rates["resources"]["search"]["/search/tweets"]["remaining"]
remaining_searches = 5
count = 1000
statuses = []
# See https://dev.twitter.com/docs/api/1.1/get/search/tweets
for i in range(remaining_searches):
    search_results = twitter_api.search.tweets(q=q, count=count)

    statuses += search_results['statuses']

In [80]:
outages = {}
for i in range(len(statuses)):
    if statuses[i]["user"]["location"]:
        if statuses[i]["user"]["location"] in outages:
            outages[statuses[i]["user"]["location"]] += 1
        else:
            outages[statuses[i]["user"]["location"]] = 1
            
# data = list(outages.items())
# an_array = np.array(data)
df = pd.DataFrame(list(outages.items()), columns=["Location", "Count"]).sort_values(by="Count", ascending=False)

In [81]:
df

,Location,Count
7,"Chicago, IL",20
18,"Brooklyn, NY",15
10,NJ,10
38,"Kurdistan, Europe",10
43,"New York, NY",10
...,...,...
30,Pittsburgh,5
31,"Seattle, WA",5
32,"New Jersey, USA",5
33,read pinned byf!,5


The business question that our group considered was 'Where are customers talking about our product/service?' How fast is our response time when fixing a power outage?

In our case we focused on the example of a power company attempting to gain data on where there are outages after a storm. This is useful as it potentially saves the company time and money that would be spent reviewing all power lines and improving customer satisfaction through decreased wait times.

We believed that Twitter data could be used to help power companies generate an early map of  outage locations and potentially how many people are affected. This early map would enable the company to prepare themselves for reports of power outages from those areas.  The companies would be able to filter locations by residential areas (towns and cities).

Additionally, the company will be able to obtain a rough estimate of their response time to a power outage by analyzing the frequency of tweets over time. This will allow them to further investigate the reasons behind a slow or fast response time, and improve or learn from their past experiences. They can also store all of this Twitter data in a database such as MongoDB in order to access old data and not have to constantly call the Twitter API in order to maintain past fix times. This will allow the company to see how their response time to fix power outages changes over time.

An argument against this project  is that most if not all power companies already have ways for you to report outages through their websites. From personal experience however, it is clear that those web sites are designed for desktop use and can be unwieldy on phones. Additionally some require customer identification which not everyone would have memorized. Sending a single location-inclusive tweet is by far the quicker, simpler solution and takes advantage of what many do when their power goes out, complain about it on Twitter. This will also allow the company to adapt to newer technologies that younger generations may prefer instead of emailing or calling.

The usage of tweets to predict the location and/or magnitude of an outage isn’t a perfect solution. Since the location attached to a tweet is almost always a nearby city instead of a specific neighborhood, tweets from densely populated areas would not give enough information to determine the exact power line that is down. If the tweet is from a sparsely populated area that has fewer power lines, however, this may give enough information to determine which power line(s) are down.


305.78343391418457

*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 15 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu).

#### We auto-process the submissions so make sure your subject line is *exactly*:

### DS3010 Case Study 1 Team ??

#### where ?? is your team number.
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

**Totoal Points: 100**


---------------------------------------------------------------------------
**Notebook results:**
    Points: 80


    -----------------------------------
    Question 1:
    Points: 20
    -----------------------------------
    
    -----------------------------------
    Question 2:
    Points: 20
    -----------------------------------
        
    -----------------------------------
    Question 3:
    Points: 20
    -----------------------------------
  
    -----------------------------------
    Question 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Come up with a business question and describe how Twitter data and a NoSQL database such as MongoDB call help you anser it.


---------------------------------------------------------------------------
**Slides (for 10 minutes of presentation): Story-telling**
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 


